In [11]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import Omniglot
from torchvision.transforms import transforms
from torchvision import models
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [12]:
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),                                                                                        
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

ds = Omniglot(root=".", download=True, transform=transform)

Files already downloaded and verified


In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()
model.to(device)

features = []
labels = []


dl = DataLoader(dataset=ds, batch_size=32, shuffle=False)

model.eval()
with torch.no_grad():
    for images, targets in dl:
        images = images.to(device)
        targets = targets.to(device)
        
        output = model(images)
        features.extend(output.cpu().numpy())
        labels.extend(targets.cpu().numpy())

trn_features, val_features, trn_labels, val_labels = train_test_split(features, labels, test_size=0.25, random_state=42)

trn_features = np.array(trn_features)
val_features = np.array(val_features)

train_features_reshaped = trn_features.reshape(
    trn_features.shape[0], 1, trn_features.shape[1]
)
test_features_reshaped = val_features.reshape(
    val_features.shape[0], 1, val_features.shape[1]
)

clf = svm.SVC(kernel="linear", C=1.0)
clf.fit(trn_features, trn_labels)
prediction = clf.predict(val_features)

/home/nekozo/newSpace/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nekozo/newSpace/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
acc = accuracy_score(val_labels, prediction)
print(val_labels[2], prediction[2])
for i in range(len(val_labels)):
    if val_labels[i] == prediction[i]:
        print(i)
        print(val_labels[i])
        break

print(f"Accuracy: {acc}")

242 242
2
242
Accuracy: 0.6159751037344399
